This notebook uses a set of functions and module within the bispy package.

GAP records are retrieved and cached based on valid/accepted ITIS species names and the invalid/not accepted ITIS species names.

In [1]:
#Import needed packages
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed

gap = bispy.gap.Gap()
bis_utils = bispy.bis.Utils()

In [2]:
# Open up the cached ITIS species list from Consult with ITIS.ipynb
with open("cache/itis.json", "r") as f:
    wlci_species = json.loads(f.read())

In [3]:
#Create list containing sciencitific names, ITIS usage, kingdom, and biological taxonomy
itis_explore = list()
for itis_doc_set in wlci_species:
    for itis_doc in itis_doc_set["itis_data"]:
        itis_explore.append({"scientific_name": itis_doc["nameWInd"],'itis_usage':itis_doc['usage'],"kingdom": itis_doc["kingdom"], 'biological_taxonomy': itis_doc["biological_taxonomy"]})     

In [4]:
itis_explore

[{'scientific_name': 'Setophaga discolor',
  'itis_usage': 'valid',
  'kingdom': 'Animalia',
  'biological_taxonomy': [{'rank': 'Kingdom', 'name': 'Animalia'},
   {'rank': 'Subkingdom', 'name': 'Bilateria'},
   {'rank': 'Infrakingdom', 'name': 'Deuterostomia'},
   {'rank': 'Phylum', 'name': 'Chordata'},
   {'rank': 'Subphylum', 'name': 'Vertebrata'},
   {'rank': 'Infraphylum', 'name': 'Gnathostomata'},
   {'rank': 'Superclass', 'name': 'Tetrapoda'},
   {'rank': 'Class', 'name': 'Aves'},
   {'rank': 'Order', 'name': 'Passeriformes'},
   {'rank': 'Family', 'name': 'Parulidae'},
   {'rank': 'Genus', 'name': 'Setophaga'},
   {'rank': 'Species', 'name': 'Setophaga discolor'}]},
 {'scientific_name': 'Pascopyrum smithii',
  'itis_usage': 'accepted',
  'kingdom': 'Plantae',
  'biological_taxonomy': [{'rank': 'Kingdom', 'name': 'Plantae'},
   {'rank': 'Subkingdom', 'name': 'Viridiplantae'},
   {'rank': 'Infrakingdom', 'name': 'Streptophyta'},
   {'rank': 'Superdivision', 'name': 'Embryophyta'},

In [5]:
# Gap only models mammals, birds, reptiles, and amphibians. Create list restricted the kingdom Animalia
possible_gap_species=[e for e in itis_explore if e['kingdom'] not in ['Plantae']]


In [6]:
possible_gap_species

[{'scientific_name': 'Setophaga discolor',
  'itis_usage': 'valid',
  'kingdom': 'Animalia',
  'biological_taxonomy': [{'rank': 'Kingdom', 'name': 'Animalia'},
   {'rank': 'Subkingdom', 'name': 'Bilateria'},
   {'rank': 'Infrakingdom', 'name': 'Deuterostomia'},
   {'rank': 'Phylum', 'name': 'Chordata'},
   {'rank': 'Subphylum', 'name': 'Vertebrata'},
   {'rank': 'Infraphylum', 'name': 'Gnathostomata'},
   {'rank': 'Superclass', 'name': 'Tetrapoda'},
   {'rank': 'Class', 'name': 'Aves'},
   {'rank': 'Order', 'name': 'Passeriformes'},
   {'rank': 'Family', 'name': 'Parulidae'},
   {'rank': 'Genus', 'name': 'Setophaga'},
   {'rank': 'Species', 'name': 'Setophaga discolor'}]},
 {'scientific_name': 'Lasiurus cinereus semotus',
  'itis_usage': 'valid',
  'kingdom': 'Animalia',
  'biological_taxonomy': [{'rank': 'Kingdom', 'name': 'Animalia'},
   {'rank': 'Subkingdom', 'name': 'Bilateria'},
   {'rank': 'Infrakingdom', 'name': 'Deuterostomia'},
   {'rank': 'Phylum', 'name': 'Chordata'},
   {'r

In [5]:
# Use joblib to run multiple requests for GAP records in parallel via kingdom Animalia scientific names
gap_results = Parallel(n_jobs=8)(delayed(gap.gap_species_search)(r["scientific_name"]) for r in possible_gap_species)
len(gap_results)

118

In [6]:
# Filter to give just cases where GAP species names matched with kingdom Animalia ITIS names
gap_success=[i for i in gap_results if i['processing_metadata']['status'] == 'success']
len(gap_success)

67

In [7]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/gap.json", gap_success))

{'Doc Cache File': 'cache/gap.json',
 'Number of Documents in Cache': 67,
 'Document Number 55': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T20:35:20.011534',
   'status_message': 'Exact Match',
   'api': "https://www.sciencebase.gov/catalog/items?parentId=527d0a83e4b0850ea0518326&format=json&fields=identifiers,files,webLinks,distributionLinks,dates&filter=itemIdentifier%3D{'key': 'Microtus californicus'}"},
  'parameters': {'Scientific Name': 'Microtus californicus'},
  'GAP Species': {'GAP Habitat Map Item': 'https://www.sciencebase.gov/catalog/item/58fa635fe4b0b7ea545257a1',
   'GAP Range Map Item': 'https://www.sciencebase.gov/catalog/item/59f5eba6e4b063d5d307e04f',
   'GAP Habitat Map WMS': 'https://www.sciencebase.gov/geoserver/CONUS_HabMap_2001/wms?service=WMS&version=1.1.0&request=GetCapabilities',
   'GAP Modeling Database Parameters URL': 'https://www.sciencebase.gov/catalog/file/get/58fa635fe4b0b7ea545257a1?f=__disk__ca%2F1f%2F54%2Fca1f5427

Check to see if any invalid/not accepted ITIS species names matched with GAP records

In [8]:
#List of kingdom Animalia invalid/not accepted ITIS species names
invalid_species_name=[i for i in possible_gap_species if i['itis_usage'] == 'invalid']
invalid_species_name.extend([i for i in possible_gap_species if i['itis_usage'] == 'not accepted'])

In [9]:
# Use joblib to run multiple requests for GAP records in parallel via kingdom Animalia invalid/not accepted ITIS scientific names
invalid_gap_results = Parallel(n_jobs=8)(delayed(gap.gap_species_search)(r["scientific_name"]) for r in invalid_species_name)

In [10]:
# Filter to give just cases where GAP species names matched with kingdom Animalia invalid/not accepted ITIS species names
invalid_result=[i for i in invalid_gap_results if i['processing_metadata']['status'] == 'success']
len(invalid_result)

1

In [11]:
#Display result to identify the GAP record that matched the invalid/not accepted ITIS name
invalid_result

[{'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T20:35:20.011534',
   'status_message': 'Exact Match',
   'api': "https://www.sciencebase.gov/catalog/items?parentId=527d0a83e4b0850ea0518326&format=json&fields=identifiers,files,webLinks,distributionLinks,dates&filter=itemIdentifier%3D{'key': 'Amphispiza belli'}"},
  'parameters': {'Scientific Name': 'Amphispiza belli'},
  'GAP Species': {'GAP Habitat Map Item': 'https://www.sciencebase.gov/catalog/item/58fa5693e4b0b7ea545253c9',
   'GAP Range Map Item': 'https://www.sciencebase.gov/catalog/item/59f5ec3ae4b063d5d307e52f',
   'GAP Habitat Map WMS': 'https://www.sciencebase.gov/geoserver/CONUS_HabMap_2001/wms?service=WMS&version=1.1.0&request=GetCapabilities',
   'GAP Modeling Database Parameters URL': 'https://www.sciencebase.gov/catalog/file/get/58fa5693e4b0b7ea545253c9?f=__disk__ed%2F7d%2Fee%2Fed7deec515fd463b5a1e188123ccb1ef658c76cd',
   'GAP ITIS Information URL': 'https://www.sciencebase.gov/catalog/fil